# Playing with debias code from Conceptnet Numberbatch

This notebook pulls all of the debiasing code and dependecies out of the Conceptnet repo. Spoilers: there's a lot of dependencies... skip to the bottom.


- the repo with download links for embeddings https://github.com/commonsense/conceptnet-numberbatch
- blog post discussing bias removal https://blog.conceptnet.io/2017/04/24/conceptnet-numberbatch-17-04-better-less-stereotyped-word-vectors/
- actual debias code to play with here: https://github.com/commonsense/conceptnet5/blob/master/conceptnet5/vectors/debias.py

## Also, then use that code to debias GloVe using the same method as Numberbatch

## Util functions 

needed code from file https://github.com/commonsense/conceptnet5/blob/master/conceptnet5/languages.py

to satisfy: `from .languages import LCODE_ALIASES`

In [1]:
LCODE_ALIASES = {
    # Pretend that various Chinese languages and variants are equivalent. This
    # is linguistically problematic, but it's also very helpful for aligning
    # them on terms where they actually are the same.
    #
    # This would mostly be a problem if ConceptNet was being used to *generate*
    # Chinese natural language text.
    'cmn': 'zh',
    'yue': 'zh',
    'zh_tw': 'zh',
    'zh_cn': 'zh',
    'zh-tw': 'zh',
    'zh-cn': 'zh',

    'nds-de': 'nds',
    'nds-nl': 'nds',

    # An easier case: consider Bahasa Indonesia and Bahasa Malay to be the
    # same language, with code 'ms', because they're already 90% the same.
    # Many sources use 'ms' to represent the entire macrolanguage, with
    # 'zsm' to refer to Bahasa Malay in particular.
    'zsm': 'ms',
    'id': 'ms',

    # We had to make a decision here on Norwegian. Norwegian Bokmål ('nb') and
    # Nynorsk ('nn') have somewhat different vocabularies but are mutually
    # intelligible. Informal variants of Norwegian, especially when spoken,
    # don't really distinguish them. Some Wiktionary entries don't distinguish
    # them either. And the CLDR data puts them both in the same macrolanguage
    # of Norwegian ('no').
    #
    # The catch is, Bokmål and Danish are *more* mutually intelligible than
    # Bokmål and Nynorsk, so maybe they should be the same language too. But
    # Nynorsk and Danish are less mutually intelligible.
    #
    # There is no language code that includes both Danish and Nynorsk, so
    # it would probably be inappropriate to group them all together. We will
    # take the easy route of making the language boundaries correspond to the
    # national boundaries, and say that 'nn' and 'nb' are both kinds of 'no'.
    #
    # More information: http://languagelog.ldc.upenn.edu/nll/?p=9516
    'nn': 'no',
    'nb': 'no',

    # Our sources have entries in Croatian, entries in Serbian, and entries
    # in Serbo-Croatian. Some of the Serbian and Serbo-Croatian entries
    # are written in Cyrillic letters, while all Croatian entries are written
    # in Latin letters. Bosnian and Montenegrin are in there somewhere,
    # too.
    #
    # Applying the same principle as Chinese, we will unify the language codes
    # into the macrolanguage 'sh' without unifying the scripts.
    'bs': 'sh',
    'hr': 'sh',
    'sr': 'sh',
    'hbs': 'sh',
    'sr-latn': 'sh',
    'sr-cyrl': 'sh',

    # More language codes that we would rather group into a broader language:
    'arb': 'ar',   # Modern Standard Arabic -> Arabic
    'arz': 'ar',   # Egyptian Arabic -> Arabic
    'ary': 'ar',   # Moroccan Arabic -> Arabic
    'ckb': 'ku',   # Central Kurdish -> Kurdish
    'mvf': 'mn',   # Peripheral Mongolian -> Mongolian
    'tl': 'fil',   # Tagalog -> Filipino
    'vro': 'et',   # Võro -> Estonian
    'sgs': 'lt',   # Samogitian -> Lithuanian
    'ciw': 'oj',   # Chippewa -> Ojibwe
    'xal': 'xwo',  # Kalmyk -> Oirat
    'ffm': 'ff',   # Maasina Fulfulde -> Fula
}

Need the following from file https://github.com/commonsense/conceptnet5/blob/master/conceptnet5/language/english.py

to satisfy: `from conceptnet5.language.english import english_filter`

In [2]:
STOPWORDS = [
    'the', 'a', 'an'
]

DROP_FIRST = ['to']


def english_filter(tokens):
    """
    Given a list of tokens, remove a small list of English stopwords.
    """
    non_stopwords = [token for token in tokens if token not in STOPWORDS]
    while non_stopwords and non_stopwords[0] in DROP_FIRST:
        non_stopwords = non_stopwords[1:]
    if non_stopwords:
        return non_stopwords
    else:
        return tokens

Need the following from file https://github.com/commonsense/conceptnet5/blob/master/conceptnet5/uri.py

to satisfy: `from conceptnet5.uri import concept_uri`

In [3]:
def join_uri(*pieces):
    """
    `join_uri` builds a URI from constituent pieces that should be joined
    with slashes (/).
    Leading and trailing on the pieces are acceptable, but will be ignored.
    The resulting URI will always begin with a slash and have its pieces
    separated by a single slash.
    The pieces do not have `standardize_text` applied to them; to make sure your
    URIs are in normal form, run `standardize_text` on each piece that represents
    arbitrary text.
    >>> join_uri('/c', 'en', 'cat')
    '/c/en/cat'
    >>> join_uri('c', 'en', ' spaces ')
    '/c/en/ spaces '
    >>> join_uri('/r/', 'AtLocation/')
    '/r/AtLocation'
    >>> join_uri('/test')
    '/test'
    >>> join_uri('test')
    '/test'
    >>> join_uri('/test', '/more/')
    '/test/more'
    """
    joined = '/' + ('/'.join([piece.strip('/') for piece in pieces]))
    return joined


def concept_uri(lang, text, *more):
    """
    `concept_uri` builds a representation of a concept, which is a word or
    phrase of a particular language, which can participate in relations with
    other concepts, and may be linked to concepts in other languages.
    Every concept has an ISO language code and a text. It may also have a part
    of speech (pos), which is typically a single letter. If it does, it may
    have a disambiguation, a string that distinguishes it from other concepts
    with the same text.
    This function should be called as follows, where arguments after `text`
    are optional:
        concept_uri(lang, text, pos, disambiguation...)
    `text` and `disambiguation` should be strings that have already been run
    through `standardize_text`.
    This is a low-level interface. See `standardized_concept_uri` in nodes.py for
    a more generally applicable function that also deals with special
    per-language handling.
    >>> concept_uri('en', 'cat')
    '/c/en/cat'
    >>> concept_uri('en', 'cat', 'n')
    '/c/en/cat/n'
    >>> concept_uri('en', 'cat', 'n', 'feline')
    '/c/en/cat/n/feline'
    >>> concept_uri('en', 'this is wrong')
    Traceback (most recent call last):
        ...
    AssertionError: 'this is wrong' is not in normalized form
    """
    assert ' ' not in text, "%r is not in normalized form" % text
    if len(more) > 0:
        if len(more[0]) != 1:
            # We misparsed a part of speech; everything after the text is
            # probably junk
            more = []
        for dis1 in more[1:]:
            assert ' ' not in dis1,\
                "%r is not in normalized form" % dis1

    return join_uri('/c', lang, text, *more)

https://raw.githubusercontent.com/commonsense/conceptnet5/master/conceptnet5/nodes.py

needed things from this file to satisfy cascading dependency:  
#from conceptnet5.nodes import standardized_concept_uri, uri_to_label

In [4]:
"""
This module constructs URIs for nodes (concepts) in various languages. This
puts the tools in conceptnet5.uri together with functions that normalize
terms and languages into a standard form.
"""

from wordfreq import simple_tokenize

def standardize_text(text, token_filter=None):
    """
    Get a string made from the tokens in the text, joined by
    underscores. The tokens may have a language-specific `token_filter`
    applied to them. See `standardize_as_list()`.

        >>> standardize_text(' cat')
        'cat'

        >>> standardize_text('a big dog', token_filter=english_filter)
        'big_dog'

        >>> standardize_text('Italian supercat')
        'italian_supercat'

        >>> standardize_text('a big dog')
        'a_big_dog'

        >>> standardize_text('a big dog', token_filter=english_filter)
        'big_dog'

        >>> standardize_text('to go', token_filter=english_filter)
        'go'

        >>> standardize_text('Test?!')
        'test'

        >>> standardize_text('TEST.')
        'test'

        >>> standardize_text('test/test')
        'test_test'

        >>> standardize_text('   u\N{COMBINING DIAERESIS}ber\\n')
        'über'

        >>> standardize_text('embedded' + chr(9) + 'tab')
        'embedded_tab'

        >>> standardize_text('_')
        ''

        >>> standardize_text(',')
        ''
    """
    tokens = simple_tokenize(text.replace('_', ' '))
    if token_filter is not None:
        tokens = token_filter(tokens)
    return '_'.join(tokens)


def standardized_concept_uri(lang, text, *more):
    """
    Make the appropriate URI for a concept in a particular language, including
    stemming the text if necessary, normalizing it, and joining it into a
    concept URI.

    Items in 'more' will not be stemmed, but will go through the other
    normalization steps.

    >>> standardized_concept_uri('en', 'this is a test')
    '/c/en/this_is_test'
    >>> standardized_concept_uri('en', 'this is a test', 'n', 'example phrase')
    '/c/en/this_is_test/n/example_phrase'
    """
    lang = lang.lower()
    if lang in LCODE_ALIASES:
        lang = LCODE_ALIASES[lang]
    if lang == 'en':
        token_filter = english_filter
    else:
        token_filter = None
    norm_text = standardize_text(text, token_filter)
    more_text = [standardize_text(item, token_filter) for item in more
                 if item is not None]
    return concept_uri(lang, norm_text, *more_text)

#normalized_concept_uri = standardized_concept_uri
#standardize_concept_uri = standardized_concept_uri

## Two Util functions below depend on all the code above

The following block of code (particularly standardized_uri() found in the following file: 
https://raw.githubusercontent.com/commonsense/conceptnet5/master/conceptnet5/vectors/__init__.py

Needed all the code blocks above as cascading dependencies. These 2 util functions are themselves dependencies for the debias.py code

In [5]:
import re

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

#from conceptnet5.nodes import standardized_concept_uri, uri_to_label

DOUBLE_DIGIT_RE = re.compile(r'[0-9][0-9]')
DIGIT_RE = re.compile(r'[0-9]')
CONCEPT_RE = re.compile(r'/c/[a-z]{2,3}/.+')


def replace_numbers(s):
    """
    Replace digits with # in any term where a sequence of two digits appears.

    This operation is applied to text that passes through word2vec, so we
    should match it.
    """
    if DOUBLE_DIGIT_RE.search(s):
        return DIGIT_RE.sub('#', s)
    else:
        return s


def standardized_uri(language, term):
    """
    Get a URI that is suitable to label a row of a vector space, by making sure
    that both ConceptNet's and word2vec's normalizations are applied to it.

    If the term already looks like a ConceptNet URI, it will only have its
    sequences of digits replaced by #. Otherwise, it will be turned into a
    ConceptNet URI in the given language, and then have its sequences of digits
    replaced.
    """
    if not CONCEPT_RE.match(term):
        term = standardized_concept_uri(language, term)
    return replace_numbers(term)


def get_vector(frame, label, language=None):
    """
    Returns the row of a vector-space DataFrame `frame` corresponding
    to the text `text`. If `language` is set, this can take in plain text
    and normalize it to ConceptNet form. Either way, it can also take in
    a label that is already in ConceptNet form.
    """
    if frame.index[1].startswith('/c/'):  # This frame has URIs in its index
        if not label.startswith('/'):
            label = standardized_uri(language, label)
        try:
            return frame.loc[label]
        except KeyError:
            return pd.Series(index=frame.columns)
    else:
        if label.startswith('/'):
            label = uri_to_label(label)
        try:
            return frame.loc[replace_numbers(label)]
        except KeyError:
            # Return a vector of all NaNs
            return pd.Series(index=frame.columns)


def normalize_vec(vec):
    """
    L2-normalize a single vector, as a 1-D ndarray or a Series.
    """
    if isinstance(vec, pd.Series):
        return normalize(vec.fillna(0).values.reshape(1, -1))[0]
    elif isinstance(vec, np.ndarray):
        return normalize(vec.reshape(1, -1))[0]
    else:
        raise TypeError(vec)


def cosine_similarity(vec1, vec2):
    """
    Get the cosine similarity between two vectors -- the cosine of the angle
    between them, ranging from -1 for anti-parallel vectors to 1 for parallel
    vectors.
    """
    return normalize_vec(vec1).dot(normalize_vec(vec2))


def similar_to_vec(frame, vec, limit=50):
    # TODO: document the assumptions here
    # - frame and vec should be normalized
    # - frame should not be made of 8-bit ints
    if vec.dot(vec) == 0.:
        return pd.Series(data=[], index=[], dtype='f')
    similarity = frame.dot(vec)
    return similarity.dropna().nlargest(limit)


def weighted_average(frame, weight_series):
    if isinstance(weight_series, list):
        weight_dict = dict(weight_series)
        weight_series = pd.Series(weight_dict)
    vec = np.zeros(frame.shape[1], dtype='f')

    for i, label in enumerate(weight_series.index):
        if label in frame.index:
            val = weight_series[i]
            vec += val * frame.loc[label].values

    return pd.Series(data=vec, index=frame.columns, dtype='f')


## The actual debias code starts here

The lengthy GLOBAL list variables are imported from external file for brevity

In [6]:

import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import normalize


# A list of English words referring to nationalities, nations, ethnicities, and
# religions. Our goal is to prevent ConceptNet from learning insults and
# stereotypes about these classes of people.

from conceptnet_code import PEOPLE_BY_ETHNICITY, PEOPLE_BY_BELIEF


# A list of things we don't want our semantic space to learn about various
# cultures of people. This list doesn't have to be exhaustive; we're modifying
# the whole vector space, so nearby terms will also be affected.
CULTURE_PREJUDICES = [
    'illegal', 'terrorist', 'evil', 'threat',
    'dumbass', 'shithead', 'wanker', 'dickhead',
    'illiterate', 'ignorant', 'inferior',
    'good',
    'sexy', 'suave',
    'wealthy', 'poor',
    'racist', 'slavery',
    'torture', 'fascist', 'persecute',
    'fraudster', 'rapist', 'robber', 'dodgy', 'perpetrator',
]

# Numberbatch acquires a "porn bias" from the Common Crawl via GloVe.
# Because so much of the Web is about porn, words such as 'teen', 'girl', and
# 'girlfriend' acquire word associations from porn.
#
# We handle this and related problems by making an axis of words that refer to
# gender or sexual orientation, and exclude them from making associations with
# porn and sexually-degrading words.

FEMALE_WORDS = [
    'woman', 'feminine', 'female',
    'girl', 'girlfriend', 'wife', 'mother', 'sister', 'daughter',
]

MALE_WORDS = [
    'man', 'masculine', 'male',
    'boy', 'boyfriend', 'husband', 'father', 'brother', 'son'
]

ORIENTATION_WORDS = [
    'gay', 'lesbian', 'bisexual', 'trans', 'transgender'
]

AGE_WORDS = [
    'young', 'teen', 'old'
]

# FIXME - load from file
SEX_PREJUDICES = [
    'slut', 'whore', 'shrew', 'bitch', 'faggot',
    'sexy', 'fuck', 'fucked', 'fucker', 'nude', 'porn',
    'cocksucker'
]

GENDERED_WORDS = FEMALE_WORDS + MALE_WORDS

# Words identified as gender stereotypes by 10 Turkers, in Bolukbasi et al.,
# "Quantifying and Reducing Stereotypes in Word Embeddings".
# https://arxiv.org/pdf/1606.06121.pdf
GENDER_NEUTRAL_WORDS = [
    'surgeon', 'nurse', 'doctor', 'midwife',
    'paramedic', 'registered nurse', 'hummer', 'minivan',
    'karate', 'gymnastics', 'woodworking', 'quilting',
    'alcoholism', 'eating disorder', 'athlete', 'gymnast',
    'neurologist', 'therapist', 'hockey', 'figure skating',
    'architect', 'interior designer', 'chauffeur', 'nanny',
    'curator', 'librarian', 'pilot', 'flight attendant',
    'drug trafficking', 'prostitution', 'musician', 'dancer',
    'beer', 'cocktail', 'weightlifting', 'gymnastics',
    'headmaster', 'guidance counselor', 'workout', 'pilates',
    'home depot', 'jcpenney', 'carpentry', 'sewing',
    'accountant', 'paralegal', 'addiction', 'eating disorder',
    'professor emeritus', 'associate professor', 'programmer',
    'homemaker',  # now augment the conceptnet list with more words from the WEAT paper
    'science', 'technology', 'physics', 'chemistry',
    'experiment', 'astronomy', 'engineering', 'engineer',
    'poetry', 'art', 'dance', 'dancer', 'literature', 'writer',
    'novel', 'symphony', 'drama', 'sculpture', 'sculptor',
    'math', 'algebra', 'geometry', 'calculus', 'equations',
    'computation', 'numbers', 'addition',
]


def get_weighted_vector(frame, weighted_terms):
    """
    Given a list of (term, weight) pairs, get a unit vector corresponding
    to the weighted average of those term vectors.

    A simplified version of VectorSpaceWrapper.get_vector().
    """
    total = frame.iloc[0] * 0.
    n_terms_used = 0  # debug
    for term, weight in weighted_terms:
        if term in frame.index:
            n_terms_used += 1
            vec = frame.loc[term]
            total += vec * weight
            
    # debug
    print('got weighted vector for', n_terms_used, 'terms')
            
    return normalize_vec(total)


def get_category_axis(frame, category_examples):
    """
    Get a vector representing the average of several example terms, where
    the terms are specified as plain English text.
    """
    # FIXME - this is not friendly to embedding frame's that are not
    # indexed in the conceptnet style ('/c/en/example')
    return get_weighted_vector(
        frame,
        [(term, 1.) for term in category_examples]
    )


def reject_subspace(frame, vecs):
    """
    Return a modification of the vector space `frame` where none of
    its rows have any correlation with any rows of `vecs`, by subtracting
    the outer product of `frame` with each normalized row of `vecs`.
    """
    current_array = frame.copy().values
    for vec in vecs:
        if not np.isnan(vec).any():
            vec = normalize_vec(vec)
            projection = current_array.dot(vec)
            np.subtract(current_array, np.outer(projection, vec), out=current_array)

    normalize(current_array, norm='l2', copy=False)

    current_array = pd.DataFrame(current_array, index=frame.index)
    current_array.fillna(0, inplace=True)
    return current_array


def get_vocabulary_vectors(frame, vocab):
    """
    Given a vocabulary (as a list of English terms), get a sub-frame of the
    given DataFrame containing just the known vectors for that vocabulary.
    """
    # FIXME - this is not friendly to embedding frame's that are not
    # indexed in the conceptnet style ('/c/en/example')
    #uris = [standardized_uri('en', term) for term in vocab]
    #return frame.loc[uris].dropna()
    return frame.loc[vocab].dropna()


def two_class_svm(frame, pos_vocab, neg_vocab):
    """
    Given a DataFrame of word vectors, and lists of words that should be
    positive or negative examples of a given category, get a linear
    decision boundary between them (and a function that estimates the
    probability of the membership of a word in that category) using an SVM.
    """
    pos_vecs = get_vocabulary_vectors(frame, pos_vocab)
    pos_values = np.ones(pos_vecs.shape[0])
    neg_vecs = get_vocabulary_vectors(frame, neg_vocab)
    neg_values = -np.ones(neg_vecs.shape[0])
    vecs = np.vstack([pos_vecs.values, neg_vecs.values])
    values = np.concatenate([pos_values, neg_values])

    svc = svm.SVC(
        verbose=False, random_state=0, max_iter=10000, class_weight='balanced',
        probability=True, kernel='linear'
    )
    svc.fit(vecs, values)
    return svc

def infer_index_format(frame):
    """
    Given a pd.DataFrame of word vectors, determine if the index is in 
    Conceptnet format (e.g. '/c/en/example_word/pos') or just the words 
    themselves, as in GloVe or word2vec.
    
    Decision is made by taking a random sample of rows from the frame and
    testing is they start with /c/*
    """
    
    sample_size = 1000
    rand_sample = list(frame.sample(sample_size, random_state=42).index)
    cn_count = 0
    
    for ind_word in rand_sample:
        if CONCEPT_RE.match(ind_word):
            cn_count += 1
            
    if cn_count < 1:
        return 'plain'
    
    cn_fraction = cn_count / sample_size
    
    if cn_fraction > 0.98:
        return 'conceptnet'
    else:
        return 'mixed'
    
    
def confirm_input_format(ind_type, input_words):
    """
    Attempts to coerce format of example words into a format that matches the
    index of the DataFrame of word vectors.
    """
    if ind_type == 'conceptnet':
        input_words = [standardized_uri('en', term) for term in input_words]
    elif ind_type == 'plain':
        for term in input_words:
            assert not CONCEPT_RE.match(term), "%r should not be in conceptnet format" % term
    else:
        # input DataFrame of word vectors has index with mixed format
        # choose either conceptnet or plain text
        raise TypeError(ind_type)
    
    return input_words
    
def de_bias_binary(frame, pos_examples, neg_examples, left_examples, right_examples):
    """
    De-bias a distinction that is presumed - for the purposes of de-biasing -
    to form two ends of a scale. The prototypical example is male vs. female,
    where words that are not inherently defined by gender end up being "more
    male" or "more female" due to stereotypes and biases in the data.

    The goal is not to remove the distinction from every word in the system's
    vocabulary, only those where making the distinction is inappropriate. A
    gender distinction between "she" and "he" is appropriate. A gender
    distinction between "doctor" and "nurse" is inappropriate.

    This function takes in four lists of vocabulary:

    - "Positive examples": examples of words that *should* be de-biased,
      such as "doctor" and "nurse" in the case of gender.

    - "Negative examples": examples of words that *should not* be de-biased,
      such as "she" and "he".

    - "Left examples": words that define one end of the distinction to be
      de-biased, such as "man".

    - "Right examples": words that define the other end of the distinction,
      such as "woman".

    The left and right examples are probably also good negative examples:
    they appropriately represent the distinction to be made, so they should
    not be de-biased.
    """
    
    # Preprocess the input words here (either convert them to conceptnet 
    # format or not depending on the format of the DataFrame index)
    ind_type = infer_index_format(frame)
    pos_examples   = confirm_input_format(ind_type, pos_examples)
    neg_examples   = confirm_input_format(ind_type, neg_examples)
    left_examples  = confirm_input_format(ind_type, left_examples)
    right_examples = confirm_input_format(ind_type, right_examples)
    
    # Make the SVM that distinguishes positive examples (words that should
    # be de-biased) from negative examples.
    category_predictor = two_class_svm(frame, pos_examples, neg_examples)

    # The SVM can predict the probability, for each vector in the frame, that
    # it's in each class. The positive class is column 1 of this prediction.
    # This gives us a vector of how much each word in the vocabulary should be
    # de-biased.
    applicability = category_predictor.predict_proba(frame)[:, 1]

    # The bias axis is the vector difference between the average right example
    # and the average left example.
    print('getting cat axis for right_examples')
    r_axis = get_category_axis(frame, right_examples)
    print('getting cat axis for left_examples')
    l_axis = get_category_axis(frame, left_examples)
    bias_axis = r_axis - l_axis

    # Make a modified version of the space that projects the bias axis to 0.
    # Then weight each row of that space by "applicability", the probability
    # that each row should be de-biased.
    modified_component = reject_subspace(frame, [bias_axis]).mul(applicability, axis=0)

    # Make another component representing the vectors that should not be
    # de-biased: the original space times (1 - applicability).
    result = frame.mul(1 - applicability, axis=0)

    # The sum of these two components is the de-biased space, where de-biasing
    # applies to each row proportional to its applicability.
    np.add(result.values, modified_component.values, out=result.values)
    del modified_component

    # L_2-normalize the resulting rows in-place.
    normalize(result.values, norm='l2', copy=False)
    return result, applicability


def de_bias_category(frame, category_examples, bias_examples):
    """
    Remove correlations between a class of words that should have biases
    removed (category_examples) and a set of words reflecting those biases
    (bias_examples). For example, the `category_examples` may be ethnicities,
    and `bias_examples` may be stereotypes about them.

    The check for whether a word should be de-biased works like
    `de_bias_binary`, where the category words are positive examples and the
    bias words are negative examples (because the words that define the bias
    presumably should not be de-biased).

    The words that should be de-biased will have their correlations with
    each of the bias words removed.
    """
    
    # Preprocess the input words here (either convert them to conceptnet 
    # format or not depending on the format of the DataFrame index)
    ind_type = infer_index_format(frame)
    category_examples = confirm_input_format(ind_type, category_examples)
    bias_examples     = confirm_input_format(ind_type, bias_examples)
    
    # Make an SVM that distinguishes words that are in the category to be
    # de-biased from words that are not.
    category_predictor = two_class_svm(frame, category_examples, bias_examples)

    # Predict the probability of each word in the vocabulary being in the
    # category.
    applicability = category_predictor.predict_proba(frame)[:, 1]
    del category_predictor

    # Make a matrix of vectors representing the correlations to remove.
    # FIXME - this is not friendly to embedding frame's that are not
    # indexed in the conceptnet style ('/c/en/example')
    #vocab = [standardized_uri('en', term) for term in bias_examples]
    #components_to_reject = frame.loc[vocab].values
    components_to_reject = frame.loc[bias_examples].values

    # Make a modified version of the space that projects the bias vectors to 0.
    # Then weight each row of that space by "applicability", the probability
    # that each row should be de-biased.
    modified_component = reject_subspace(frame, components_to_reject).mul(applicability, axis=0)
    del components_to_reject

    # Make another component representing the vectors that should not be
    # de-biased: the original space times (1 - applicability).
    result = frame.mul(1 - applicability, axis=0)

    # The sum of these two components is the de-biased space, where de-biasing
    # applies to each row proportional to its applicability.
    np.add(result.values, modified_component.values, out=result.values)
    del modified_component

    # L_2-normalize the resulting rows in-place.
    normalize(result.values, norm='l2', copy=False)
    return result, applicability


def de_bias_frame(frame):
    """
    Take in a DataFrame representing a semantic space, and make a strong
    effort to modify it to remove biases and prejudices against certain
    classes of people.

    The resulting space attempts not to learn stereotyped associations with
    anyone's race, color, religion, national origin, sex, gender presentation,
    or sexual orientation.
    """
    newframe, app_ethn = de_bias_category(frame, PEOPLE_BY_ETHNICITY, CULTURE_PREJUDICES + SEX_PREJUDICES)
    newframe, app_cult = de_bias_category(newframe, PEOPLE_BY_BELIEF, CULTURE_PREJUDICES + SEX_PREJUDICES)
    newframe, app_misc = de_bias_category(newframe, FEMALE_WORDS + MALE_WORDS + ORIENTATION_WORDS + AGE_WORDS, CULTURE_PREJUDICES + SEX_PREJUDICES)
    newframe, app_gend = de_bias_binary(newframe, GENDER_NEUTRAL_WORDS, GENDERED_WORDS, MALE_WORDS, FEMALE_WORDS)
    return newframe, app_ethn, app_cult, app_misc, app_gend


## Now Load some data to pass to de_bias_frame()

In [27]:
def load_hdf(filename):
    """
    Load a semantic vector space from an HDF5 file.
    HDF5 is a complex format that can contain many instances of different kinds
    of data. The convention we use is that the file contains one labeled
    matrix, named "mat".
    """
    return pd.read_hdf(filename, 'mat', encoding='utf-8', dtype=np.float64)

In [28]:
cumbers = load_hdf('data/17.06.mini.h5')

In [33]:
cumbers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362891 entries, /c/de/###er to /c/zh/龟
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 833.4+ MB


In [30]:
# debiasing only works on float64 i think (what is a better way to change dtypes?
# I couldn't seem to get pandas to load these from hdf5 into anything but int8 )
for col in cumbers.columns:
    cumbers[col] = cumbers[col].astype(np.float64)
cumbers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362891 entries, /c/de/###er to /c/zh/龟
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 833.4+ MB


In [34]:
%%time
deb = de_bias_frame(cumbers)
deb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362891 entries, /c/de/###er to /c/zh/龟
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 843.4+ MB
CPU times: user 2min 53s, sys: 1min 3s, total: 3min 57s
Wall time: 3min 42s


In [35]:
# what does the index look like
ind = list(cumbers.index)
ind[200000:][:10]

['/c/fr/perçoivent',
 '/c/fr/perçu',
 '/c/fr/perçue',
 '/c/fr/perçues',
 '/c/fr/perçus',
 '/c/fr/pesant',
 '/c/fr/pesante',
 '/c/fr/pesanteur',
 '/c/fr/peser',
 '/c/fr/pessimisme']

## Now try with GloVe

In [7]:
import pandas as pd
import numpy as np
import gzip
import zipfile
import io

def file_stream(fpath, inner_path=None, encoding='utf-8'):
    """Generator for reading file objects. Handles gzipped and zip
    archives."""
    
    if fpath.endswith('.zip'):
        with zipfile.ZipFile(fpath) as zfile:
            try:
                with zfile.open(inner_path) as readfile:
                    for line in io.TextIOWrapper(readfile, 'utf-8'):
                        yield line
            except Exception as e:
                print('need a valid inner_path argument, one of these: ', 
                      zfile.namelist())
                raise e  # fixme raise proper excecption
                
    elif fpath.endswith('.gz'):
        raise 'not implemented yet'
    else:
        with open(fpath, encoding='utf-8') as infile:
            for line in (infile):
                yield line

def load_zipped_embeddings(filename, inner_path=None, 
                           encoding='utf-8', vocab_only=False):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    
    if vocab_only:
        for line in file_stream(filename, inner_path, encoding):
            if not line.startswith(vocab_only):
                continue
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue

            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    else:
        for line in file_stream(filename, inner_path, encoding):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype=np.float64)


In [8]:
%%time
#zip_path = 'data/glove.840B.300d.zip'
#zfile = 'glove.840B.300d.txt'
zip_path = 'data/glove.6B.zip'
zfile = 'glove.6B.300d.txt'
embeddings = load_zipped_embeddings(zip_path, inner_path=zfile)
print(embeddings.shape)

(400000, 300)
CPU times: user 1min 17s, sys: 2.83 s, total: 1min 20s
Wall time: 1min 20s


In [9]:
embeddings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400000 entries, the to sandberger
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 918.6+ MB


In [10]:
art_vec_biased = embeddings.loc['art']
art_vec_biased

0      0.191910
1      0.350140
2     -0.460470
3     -0.068022
4      0.666540
5     -0.023657
6     -0.046550
7     -0.595320
8     -0.229570
9     -1.144000
10     0.507740
11    -0.061745
12    -0.109020
13     0.773590
14     0.197490
15    -0.685650
16     0.100170
17     0.020734
18     0.284430
19     0.585180
20    -0.144190
21     0.305860
22     0.250030
23     0.457050
24     0.399640
25     0.017308
26    -0.296780
27     0.111690
28    -0.212350
29     0.872930
         ...   
270    0.069308
271    0.313710
272    0.709770
273   -0.344320
274   -0.119630
275    0.640050
276   -1.570500
277    0.037442
278   -0.014749
279   -0.258690
280   -0.145470
281   -0.266500
282   -0.073628
283   -0.411070
284    0.576080
285    0.305980
286   -0.359500
287    0.523640
288   -0.240350
289    0.078563
290   -0.072383
291   -0.627950
292   -0.795580
293   -0.018491
294    0.652040
295   -0.260530
296    0.001454
297   -0.172540
298    0.331540
299   -0.147650
Name: art, Length: 300, 

In [11]:
teacher_vec_biased = embeddings.loc['teacher']
teacher_vec_biased

0     -0.279320
1     -0.230860
2     -0.019087
3     -0.141660
4     -0.015263
5     -0.597030
6     -0.118750
7     -0.686740
8     -0.218320
9     -0.818380
10    -0.039471
11     0.252810
12     0.256940
13     0.281090
14     0.237340
15     0.212450
16     0.277160
17    -0.367030
18     0.061541
19    -0.237930
20    -0.201850
21    -0.139690
22    -0.828230
23     0.270940
24    -0.342580
25     0.161700
26     0.099299
27    -0.278200
28     0.471990
29     0.246180
         ...   
270   -0.438290
271    0.134150
272    0.387900
273   -0.303600
274    0.148190
275   -0.057197
276   -1.448500
277   -0.303900
278    0.043171
279    0.084293
280   -0.077757
281    0.773080
282   -0.454120
283   -0.456860
284    0.260800
285    0.774490
286   -0.004975
287    0.100650
288   -0.494700
289   -0.052624
290    0.132570
291   -0.288320
292   -0.138510
293   -0.152430
294    0.138840
295    0.070689
296    0.115670
297   -0.150690
298   -0.360910
299    0.543410
Name: teacher, Length: 3

In [12]:
%%time
glove_deb, app_ethn, app_cult, app_misc, app_gend = de_bias_frame(embeddings)
glove_deb.info()

getting cat axis for right_examples
got weighted vector for 9 terms
getting cat axis for left_examples
got weighted vector for 9 terms
<class 'pandas.core.frame.DataFrame'>
Index: 400000 entries, the to sandberger
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 928.6+ MB
CPU times: user 3min 47s, sys: 1min 7s, total: 4min 54s
Wall time: 4min 39s


#### Did anything actually change?

In [13]:
art_vec = glove_deb.loc['art']
art_vec

0      0.045964
1      0.054183
2     -0.080615
3     -0.005628
4      0.117184
5     -0.016679
6     -0.003924
7     -0.092368
8     -0.053686
9     -0.106373
10     0.066714
11    -0.013379
12    -0.007837
13     0.110635
14     0.045192
15    -0.093920
16     0.012574
17     0.001569
18     0.063335
19     0.079881
20    -0.001634
21     0.026795
22     0.002635
23     0.037535
24     0.078586
25     0.011516
26    -0.039343
27     0.033783
28    -0.035886
29     0.135465
         ...   
270   -0.004987
271    0.054422
272    0.126407
273   -0.053295
274   -0.000271
275    0.083734
276   -0.148871
277   -0.010596
278   -0.020537
279   -0.039663
280   -0.033425
281   -0.035942
282   -0.001187
283   -0.068091
284    0.095069
285    0.037435
286   -0.050581
287    0.044748
288   -0.053254
289    0.006982
290    0.002386
291   -0.091317
292   -0.099861
293   -0.021328
294    0.099760
295   -0.028694
296   -0.013165
297   -0.026832
298    0.059102
299   -0.013004
Name: art, Length: 300, 

In [14]:
teacher_vec = glove_deb.loc['teacher']
teacher_vec

0      0.019862
1     -0.049448
2     -0.010979
3      0.007938
4     -0.031143
5     -0.124095
6     -0.025106
7     -0.122940
8     -0.036864
9     -0.013698
10    -0.055004
11     0.087683
12     0.079068
13     0.021258
14     0.034430
15     0.070830
16     0.055656
17    -0.052178
18     0.033013
19    -0.009732
20     0.028755
21    -0.041840
22    -0.166747
23     0.016043
24    -0.040497
25     0.051217
26     0.002986
27    -0.040750
28     0.062175
29     0.076459
         ...   
270   -0.066859
271    0.046716
272    0.052156
273   -0.036071
274    0.016940
275   -0.009257
276   -0.061543
277   -0.108731
278   -0.019918
279   -0.010785
280    0.000110
281    0.119338
282   -0.062417
283   -0.108852
284    0.057933
285    0.091050
286   -0.023540
287   -0.016241
288   -0.102136
289    0.001571
290    0.048383
291   -0.015060
292    0.020030
293   -0.044938
294    0.026948
295    0.009422
296    0.028973
297   -0.006327
298   -0.034961
299    0.085105
Name: teacher, Length: 3

In [15]:
app_ethn[:10]

array([ 0.94771977,  0.94006975,  0.64661825,  0.95850904,  0.41772661,
        0.82307801,  0.9917094 ,  0.37108394,  0.00821813,  0.4261017 ])

In [21]:
# number of vectors significantly changed for ethnicity
changed = 0
for v in app_ethn > 0.8:
    if v == True:
        changed += 1
print(changed)

230921


In [29]:
# number of vectors significantly changed for culture
changed = 0
for v in app_cult > 0.4:
    if v == True:
        changed += 1
print(changed)

29346


In [36]:
# number of vectors significantly changed for misc groups
changed = 0
for v in app_misc > 0.4:
    if v == True:
        changed += 1
print(changed)

5982


In [32]:
# number of vectors significantly changed for gender
changed = 0
for v in app_gend > 0.9:
    if v == True:
        changed += 1
print(changed)

395542


#### Save debiased embedding to file

In [12]:
# save to file
#glove_deb.to_pickle('data/glove.6B.300d_debiased.pkl')

In [13]:
# still need to add a size header line to this file (#words, #dimensions)
glove_deb.to_csv('data/glove.6B.300d_debiased.txt', 
                 sep=' ', encoding='utf-8', header=False )

In [51]:
# FIXME - need to specify encoding='utf8' somehow
out_path = 'data/glove.6B.300d_debiased_w2vec_fmt.txt'
with open(out_path, 'w', encoding='utf8') as f:
    np.savetxt(f, 
               glove_deb.reset_index().values, 
               delimiter=" ", 
               header="{} {}".format(len(glove_deb), len(glove_deb.columns)),
               comments="",
               fmt=["%s"] + ["%.18e"]*len(glove_deb.columns))

TypeError: write() argument must be str, not bytes